In [1]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import multiprocessing
import os
import glob

In [2]:
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, ".."))
MODEL_PATH = os.path.join(project_root, "ai_model_dir", "gguf", "hyperclova", "hyperclova-seed-text-1.5b-q4-k-m.gguf")
EMBEDDING_MODEL_PATH = os.path.join(project_root, "ai_model_dir", "hf", "BGE-m3-ko")

In [9]:
# base_system_prompt.txt 로드
with open("../prompts/system/base_system_prompt.txt", "r", encoding="utf-8") as f:
    base_system_prompt = f.read()

# qa_prompt.txt 로드 
with open("../prompts/tasks/qa_prompt.txt", "r", encoding="utf-8") as f:
    qa_prompt = f.read()

In [4]:
embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_PATH,
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

C:\Users\jw160\AppData\Local\Temp\ipykernel_11332\3960392107.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [5]:
pdf_path = os.path.join(project_root, "data")

In [6]:
pdf_files = glob.glob(os.path.join(pdf_path, "*.pdf"))

In [7]:
documents = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    documents.extend(loader.load())

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.split_documents(documents)

# FAISS 벡터 저장소 생성 
vectorstore = FAISS.from_documents(texts, embeddings)

# 로컬에 저장
vectorstore.save_local("./faiss_index")



In [33]:
vectorstore = FAISS.load_local("./faiss_index", embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", 
    search_kwargs={
        "k": 5,
        "score_threshold": 0.7,
        "filter": {'paper_title':'인공지능 기본법'}
    }
)

In [30]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = LlamaCpp(
    model_path=MODEL_PATH,      # 모델 파일 경로
    temperature=0.8,            # 생성 텍스트의 무작위성 조절 (0~1)
    max_tokens=512,             # 생성할 최대 토큰 수
    top_p=1,                    # 누적 확률 기반 샘플링을 위한 임계값
    callback_manager=callback_manager,  # 콜백 매니저 설정
    verbose=True,               # 상세 로그 출력 여부
    n_ctx=2048,                # 컨텍스트 길이 (입력+출력 토큰의 최대 길이)
    n_threads=multiprocessing.cpu_count() - 1,  # 사용할 CPU 스레드 수 (전체 코어 수 - 1)
)

llama_model_loader: loaded meta data with 33 key-value pairs and 218 tensors from c:\Users\jw160\project\ai_management\ai_model_dir\gguf\hyperclova\hyperclova-seed-text-1.5b-q4-k-m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Hyperclova Seed Text 1.5b
llama_model_loader: - kv   3:                           general.basename str              = hyperclova-seed-text
llama_model_loader: - kv   4:                         general.size_label str              = 1.5B
llama_model_loader: - kv   5:                            general.license str              = other
llama_model_loader: - kv   6:                       general.license

llama_model_loader: - kv  17:               llama.attention.value_length u32              = 128
llama_model_loader: - kv  18:                           llama.vocab_size u32              = 110592
llama_model_loader: - kv  19:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv  20:                       tokenizer.ggml.model str              = gpt2
llama_model_loader: - kv  21:                         tokenizer.ggml.pre str              = dbrx
llama_model_loader: - kv  22:                      tokenizer.ggml.tokens arr[str,110592]  = ["!", "\"", "#", "$", "%", "&", "'", ...
llama_model_loader: - kv  23:                  tokenizer.ggml.token_type arr[i32,110592]  = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
llama_model_loader: - kv  24:                      tokenizer.ggml.merges arr[str,110305]  = ["Ġ Ġ", "ĠĠ ĠĠ", "i n", "Ġ t",...
llama_model_loader: - kv  25:                tokenizer.ggml.bos_token_id u32              = 100257
llama_model_loader: - kv

In [34]:
template=f"{base_system_prompt}\n{{context}}\n\n{qa_prompt}"
prompt_template = PromptTemplate.from_template(template=template, template_format="f-string")

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,                     # llama-cpp나 OpenAI 등 langchain-compatible LLM
    retriever=retriever,         # langchain-compatible retriever
    chain_type="stuff",          # "stuff", "map_reduce", "refine" 중 선택
    chain_type_kwargs={"prompt": prompt_template}  # PromptTemplate을 전달
)

In [35]:
rag_chain.invoke("인공지능 기본법에 대해 구체적으로 설명").get("result")

No relevant docs were retrieved using the relevance score threshold 0.6


 인공지능

Llama.generate: 302 prefix-match hit, remaining 1 prompt tokens to eval


 기본법에 대한 구체적인 설명은 제공되지 않았습니다.
인공지능 기본법

인공지능 기본법은 국가의 인공지능 기술 발전과 보장을 위해 제정하는 법입니다.

인공지능 기본법은 다음과 같은 내용을 담고 있습니다:

1. 인공지능의 개발·보유·이용 및 판매에 관한 규제
2. 인공지능에 대한 연구·개발·지원 및 판매에 관한 규제에 대한 규제
3. 인공지능에 대한 연구·개발·지원 및 판매에 관한 규제에 대한 규제

4. 인공지능에 대한 연구·개발·지원 및 판매에 관한 규제에 대한 규제

5. 인공지능에 대한 연구·개발·지원 및 판매에 관한 규제에 대한 규제


아울러 제게 AI에 대해 기본, 기본, 기본, 기본, 기본법
이
아
다
내
하
위
제
게 AI 
인공지능 기본법에 대해 구체적으로 설명해 주실 수 있나요?"

아울러 제게 AI에 대해 기본, 기본, 기본, 기본, 기본법
이
아
다
내
하
위
제
게 AI 
인공지능 기본법에 대해 구체적으로 설명해 주실 수 있나요

KeyboardInterrupt: 